In [22]:
import nltk
import string

In [23]:
stemmer=nltk.stem.snowball.GermanStemmer()
stemmer=nltk.stem.SnowballStemmer('german')

In [24]:
training_data = []
training_data.append({"class":"greeting", "sentence":"Wie geht 's?"})
training_data.append({"class":"greeting", "sentence":"Wie fühlst Du Dich?"})
training_data.append({"class":"greeting", "sentence":"Guten Tag."})
training_data.append({"class":"greeting", "sentence":"Wie geht 's Dir heute?"})
training_data.append({"class":"greeting", "sentence":"Wie läufts?"})
training_data.append({"class":"greeting", "sentence":"Hallo?"})

training_data.append({"class":"goodbye", "sentence":"Tschüss."})
training_data.append({"class":"goodbye", "sentence":"Bis dann."})
training_data.append({"class":"goodbye", "sentence":"Bis bald."})
training_data.append({"class":"goodbye", "sentence":"Auf Wiedersehen"})

training_data.append({"class":"weathertoday", "sentence":"wie wird das wetter heute"})
training_data.append({"class":"weathertoday", "sentence":"wie wird das wetter heute in Dresden"})
training_data.append({"class":"weathertoday", "sentence":"wie wird das wetter heute in Sachsen"})
training_data.append({"class":"weathertoday", "sentence":"wie ist das wetter"})
training_data.append({"class":"weathertoday", "sentence":"wie ist das wetter in Dresden"})
training_data.append({"class":"weathertoday", "sentence":"wie ist das wetter in Sachsen"})

training_data.append({"class":"weathertomorrow", "sentence":"wie wird das wetter morgen"})
training_data.append({"class":"weathertomorrow", "sentence":"wie wird das wetter morgen in Dresden"})
training_data.append({"class":"weathertomorrow", "sentence":"wie wird das wetter morgen in Sachsen"})
training_data.append({"class":"weathertomorrow", "sentence":"wie wird das wetter"})
training_data.append({"class":"weathertomorrow", "sentence":"wie wird das wetter in Dresden"})
training_data.append({"class":"weathertomorrow", "sentence":"wie wird das wetter in Sachsen"})

In [25]:
# capture unique stemmed words in the training corpus
corpus_words = {}
class_words = {}
# turn a list into a set (of unique items) and then a list again (this removes duplicates)
classes = list(set([a['class'] for a in training_data]))
for c in classes:
    # prepare a list of words within each class
    class_words[c] = set()

# loop through each sentence in our training data
for data in training_data:
    # tokenize each sentence into words
    for word in nltk.tokenize.WordPunctTokenizer().tokenize(text=data['sentence']):
        # ignore a some things
        if word not in ["?", "'s"]:
            # stem and lowercase each word
            stemmed_word = stemmer.stem(word.lower())
            # have we not seen this word already?
            if stemmed_word not in corpus_words:
                corpus_words[stemmed_word] = 1
            else:
                corpus_words[stemmed_word] += 1

            # add the word to our words in class list
            class_words[data['class']].add(stemmed_word)

In [26]:
# calculate a score for a given class taking into account word commonality
def calculate_class_score(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    tokens=nltk.tokenize.WordPunctTokenizer().tokenize(sentence)
    tokens = [i for i in tokens if i not in string.punctuation]
    for word in tokens:
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with relative weight
            score += (1 / corpus_words[stemmer.stem(word.lower())])

            if show_details:
                print ("   match: %s (%s)" % (stemmer.stem(word.lower()), 1 / corpus_words[stemmer.stem(word.lower())]))
    return score

In [27]:
# return the class with highest score for sentence
def classify(sentence):
    high_class = None
    high_score = 0
    # loop through our classes
    for c in class_words.keys():
        # calculate score of sentence for each class
        score = calculate_class_score(sentence, c, show_details=False)
        # keep track of highest score
        if score > high_score:
            high_class = c
            high_score = score

    return high_class, high_score